In [ ]:
from langchain import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(
    openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0,
    model_name='text-davinci-003'
)

In [ ]:
# Next we wiil initialize a calculator tool using an LLMMathChain

from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm, verbose=True)



#intitialize the math tool

math_tool = Tool(
    name='calculator',
    func=llm_math.run,
    description="Userful for when you need to answer questions about math"
)

# When giving tools to LLM, we must pass as list of tools

tools = [math_tool]

In [ ]:
tools[0].name, tools[0].description

In [ ]:
from langchain.agents import load_tools

tools = load_tools(['llm-math'],llm=llm)

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools, 
    llm = llm,
    verbose= True,
    max_iterations = 3
)

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

In [ ]:
(4.5*2.1)**2.2

In [ ]:
# Lets try this with zero shot agent
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")

In [ ]:
# But what happens when we ask it a question from non math

zero_shot_agent("what is the capital of Norway?")


#Clearly calculator wont work for this
#Because the agent is forcing us to use the calculator tool which is not applicable at all      

In [ ]:
# So as a workaround we can implement is to just add another tool that can answer this question


from langchain.prompts import PromptTemplate
from langchain.chains import  LLMChain


prompt = PromptTemplate(
    input_variables=["query"],
    template = "{query}"

)

llm_chain = LLMChain(llm=llm, prompt=prompt)


#initialize the LLM tool

llm_tool = Tool(
    name = "Language Model",
    func = llm_chain.run,
    description="use this tool for general purpose queries and logic"
)

In [ ]:
#append the tool

tools.append(llm_tool)

In [ ]:
#Now initialize the agent with our two tools

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)


In [ ]:
zero_shot_agent("Who designed the Victoria Memorial, Kolkata?Find the square of the age of the person when he designed the Victoria Memorial, Kolkata")

In [ ]:
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    openai_api_base=os.environ['OPENAI_REVERSE_PROXY'],
    temperature=0
)

In [ ]:

from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

# With all of that set up, lets jump into Agents.

## What is an agent?

In [39]:
# We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.


from sqlalchemy import MetaData

metadata_obj = MetaData()

In [ ]:
from sqlalchemy import Column, Integer, String, Table, Date, Float


stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [ ]:
from sqlalchemy import  create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [ ]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [ ]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id = obs[0],
    stock_ticker = obs[1],
    price = obs[2],
    date = obs[3]
    )


    with engine.begin() as conn:
        conn.execute(stmt)

In [ ]:
for obs in observations:
    insert_obs(obs)

In [ ]:
!pip install langchain_experimental

In [ ]:
from langchain.chat_models import ChatVertexAI
llm = ChatVertexAI(temperature=0)

In [ ]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
db = SQLDatabase(engine=engine)

In [ ]:
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
from langchain.agents import Tool
sql_tool  = Tool(
    name='Stock DB',
    func = sql_chain.run,
    description="Userful for when you need to answer questions about stocks and their prices"
)

Agent types

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [ ]:
tools.append(sql_tool)

In [10]:
# As the name suggests we will use this agent to perform zero shot tasks on the input.In other words, this agent will have no memory.


from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [ ]:
#Now lets check it out

result = count_tokens(
    zero_shot_agent, 
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " + 
    "between the same stock prices in January the 4th?"
)

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

## Agent type 2: Conversational React

In [12]:
from langchain import OpenAI
from langchain.agents import load_tools
import os
import dotenv


llm = OpenAI(
    temperature = 0,
    openai_api_base = dotenv.get_key('.env','OPENAI_REVERSE_PROXY'),
    openai_api_key = dotenv.get_key('.env','OPENAI_API_KEY')
)

In [13]:
tools =  load_tools(
    ['llm-math'],
    llm=llm
)

In [14]:
tools.append(sql_tool)

NameError: name 'sql_tool' is not defined

In [15]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [16]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    
)


In [ ]:
result= count_tokens(
    conversational_agent,
    "Please provide me the stock prices for ABC on January the 1st"
)

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.template)

In [ ]:
result = count_tokens(
    conversational_agent, 
    "What is the multiplication of the ratio of the prices of stocks 'ABC' and 'XYZ' in January 3rd and the ratio of the same prices of the same stocks in January the 4th?"
)

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [ ]:


docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)



In [ ]:
count_tokens(docstore_agent, "What were Archimedes' last words?")

In [17]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=os.environ['SERPER_API_KEY'])
search_tool  = Tool(
    name='Intermediate Answer',
    func = search.run,
    description = "Use this function to google search"
)

self_ask_with_search = initialize_agent([search_tool], llm, agent="self-ask-with-search", verbose=True)


In [18]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [19]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [20]:
tools.append(search_tool)

In [21]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history')

In [25]:

conversational_agent = initialize_agent(
    tools, 
    llm, 
    verbose=True,
    max_iterations=30,
    memory = memory
)

In [26]:
import os
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
conversational_agent("{sys_message}:You are terrible at maths, so whenever a calculation comes always use the calculator tool,Find the square root of the nearest prime number of the age of Akbar when he died")



> Entering new AgentExecutor chain...
 I need to find the nearest prime number of Akbar's age when he died
Action: Intermediate Answer
Action Input: nearest prime number of Akbar's age when he died
Observation: Akbar ; Jalal-ud-din Muhammad Akbar 15 October 1542 · Amarkot, Rajputana (present-day Umerkot, Sindh, Pakistan) · 27 October 1605(1605-10-27) (aged 63) Fatehpur ...
Thought: I need to find the square root of 63
Action: Calculator
Action Input: 63
Observation: Answer: 63
Thought: I now know the final answer
Final Answer: 8

> Finished chain.


{'input': '{sys_message}:You are terrible at maths, so whenever a calculation comes always use the calculator tool,Find the square root of the nearest prime number of the age of Akbar when he died',
 'chat_history': 'Human: Find the square root of the nearest prime number of the age of Akbar when he died\nAI: Agent stopped due to iteration limit or time limit.\nHuman: Find the square root of the nearest prime number of the age of Akbar when he died\nAI: The square root of the nearest prime number of the age of Akbar when he died is 8.\nHuman: {sys_message}:You are terrible at maths, so whenever a calculation comes always use the calculator tool,Find the square root of the nearest prime number of the age of Akbar when he died\nAI: 8\nHuman: find the square root of 63\nAI: 8',
 'output': '8'}

In [37]:
conversational_agent("""Find the hierarcy of the Mughal empire.Find the name of the rulers and the time period they reigned, then create an sql query to create a table which has three columns one for ruler name and one for time period and one for the time in years they ruled and fill the values with the rulers' names and time period and the time they ruled
"EXAMPLE":
|RULER_NAME|TIME_PERIOD|
|Mughal Empire|1947-1966|
|Mughal Empire|1966-1971|
|Mughal Empire|1971-1981|
|
"""
)



> Entering new AgentExecutor chain...
 I need to find the names of the rulers and the time period they reigned.
Action: Intermediate Answer
Action Input: "Mughal Empire rulers and time period"
Observation: The Mughal Empire's first ruler was Babur. On April 20, 1526, Babur established his kingdom after defeating Ibrahim Lodi in the first battle of Panipat (1526). However, the kingdom's decline began with the sixth emperor Aurangzeb, and it was finally overthrown on September 21, 1857, as a result of the Revolt of 1857.
Thought: I need to create an SQL query to create a table with the rulers' names and time period.
Action: Intermediate Answer
Action Input: "Mughal Empire rulers and time period SQL query"
Observation: Mughal emperors were the supreme heads of state of the Mughal Empire on the Indian subcontinent, mainly corresponding to the modern countries of India, ...
Thought: I need to find the time in years each ruler reigned.
Action: Intermediate Answer
Action Input: "Mughal Empi

{'input': 'Find the hierarcy of the Mughal empire.Find the name of the rulers and the time period they reigned, then create an sql query to create a table which has three columns one for ruler name and one for time period and one for the time in years they ruled and fill the values with the rulers\' names and time period and the time they ruled\n"EXAMPLE":\n|RULER_NAME|TIME_PERIOD|\n|Mughal Empire|1947-1966|\n|Mughal Empire|1966-1971|\n|Mughal Empire|1971-1981|\n|\n',
 'chat_history': 'Human: Find the square root of the nearest prime number of the age of Akbar when he died\nAI: Agent stopped due to iteration limit or time limit.\nHuman: Find the square root of the nearest prime number of the age of Akbar when he died\nAI: The square root of the nearest prime number of the age of Akbar when he died is 8.\nHuman: {sys_message}:You are terrible at maths, so whenever a calculation comes always use the calculator tool,Find the square root of the nearest prime number of the age of Akbar when

In [44]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///mydb.db")

In [47]:
conn = engine.connect()

In [52]:
conn.execute(sqlalchemy.text("CREATE TABLE mughal_empire (ruler_name VARCHAR(50),time_period VARCHAR(50),time_in_years INT);"))

In [54]:
conn.execute(sqlalchemy.text("INSERT INTO mughal_empire (ruler_name, time_period, time_in_years) VALUES ('Babur', '1483-1531', 48), ('Humayun', '1508-1556', 48), ('Akbar', '1542-1605', 63), ('Jahangir', '1569-1627', 58),('Shah Jahan', '1628-1658', 30),('Aurangzeb', '1658-1707', 49);"))

In [57]:
conn.close()